In [ ]:
#Supercell generator: C2db

import os
from ase.io import write
from ase.db import connect
from ase.build import make_supercell
import numpy as np

# Define the transformation matrix for the supercell expansion
# For example, 2x2 replication in the a and b directions (2D plane)
transformation_matrix = np.array([[SUPERCELL_SIZE, 0, 0],
                                  [0, SUPERCELL_SIZE, 0],
                                  [0, 0, 1]])  # No replication in the c-direction

# Create a folder for saving the supercell files
output_folder = 'Supercells/supercells_c2db'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Connect to the C2DB database (assuming it's in the current directory)
db = connect('Materials/c2db.db')

# Iterate over the structures in the database
for row in db.select():
    # Read the structure (unit cell) from the database
    unit_cell = row.toatoms()

    # Create the supercell (repeated structure)
    supercell = make_supercell(unit_cell, transformation_matrix)

    # Define a filename based on the material's ID in the database
    filename = os.path.join(output_folder, f"supercell_{row.id}.xyz")

    # Write the supercell structure to an XYZ file inside the 'supercells' folder
    write(filename, supercell)

    print(f"Supercell expansion complete for {row.id} and saved to '{filename}'")

   


print(f"Supercell expansion for the all materials in the database is complete and saved in the '{output_folder}' folder.")

In [ ]:
#Supercell generator: flatband

import os
import numpy as np
from ase.io import write
from ase.build import make_supercell
from ase import Atoms
import ast  # Use ast.literal_eval for safer evaluation

SUPERCELL_SIZE = 1

transformation_matrix = np.array([[SUPERCELL_SIZE, 0, 0],
                                  [0, SUPERCELL_SIZE, 0],
                                  [0, 0, 1]])  # No replication in the c-direction

# Create a folder for saving the supercell files
output_folder = 'Supercells/supercells_flatband_1x1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to parse the structures from the text file
def parse_structures(filename):
    import re
    materials = []
    with open(filename, 'r') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('2dm-'):
            material = {}
            material['id'] = line.strip()
            i += 1
            # Read formula and code
            line = lines[i].strip()
            formula_and_code = line.split()
            material['formula'] = formula_and_code[0]
            material['some_code'] = formula_and_code[1] if len(formula_and_code) > 1 else ''
            i += 1
            # Read lattice vectors
            if lines[i].strip() == 'lattice vector':
                i += 1
                material['lattice_vectors'] = ast.literal_eval(lines[i].strip())
                i += 1
            else:
                raise ValueError(f"Expected 'lattice vector' at line {i+1}")
            # Skip empty lines
            while i < len(lines) and not lines[i].strip():
                i += 1
            # Read headers
            if i < len(lines):
                headers = lines[i].strip().split()
                i += 1
            else:
                break
            # Read atom data
            atoms = []
            while i < len(lines):
                line = lines[i].strip()
                if not line or line.startswith('2dm-'):
                    break
                # Parse the atom line
                try:
                    atom_dict = parse_atom_line(line)
                    atoms.append(atom_dict)
                except ValueError as e:
                    print(f"Warning: {e}")
                i += 1
            material['atoms'] = atoms
            materials.append(material)
        else:
            i += 1
    return materials

def parse_atom_line(line):
    import re
    # Remove leading index if present
    line = line.strip()
    if line and line[0].isdigit():
        parts = line.split(None, 1)
        line = parts[1]
    # Now extract the label, which is the first word
    parts = line.split(None, 1)
    if len(parts) < 2:
        raise ValueError("Could not parse atom line: {}".format(line))
    label = parts[0]
    rest = parts[1]
    # Use regular expressions to find the content inside brackets or braces
    pattern = r'\[.*?\]|\{.*?\}|[^\s]+'
    matches = re.findall(pattern, rest)
    # The expected order: xyz, abc, species, properties
    if len(matches) >= 4:
        xyz_str = matches[0]
        abc_str = matches[1]
        species_str = matches[2]
        properties_str = matches[3]
        return {
            'label': label,
            'xyz': xyz_str,
            'abc': abc_str,
            'species': species_str,
            'properties': properties_str
        }
    else:
        raise ValueError("Could not parse atom line: {}".format(line))

# Parse the structures from the text file
materials = parse_structures('Materials/reduced_sublattice_structure.txt')

# Iterate over the parsed materials and generate supercells
for material in materials:
    # Extract lattice vectors and atom data
    cell = material['lattice_vectors']
    atoms_data = material['atoms']
    symbols = []
    positions = []
    for atom in atoms_data:
        symbol = atom['label']
        position = ast.literal_eval(atom['xyz'])  # Safely convert string to list
        symbols.append(symbol)
        positions.append(position)
    # Create the unit cell structure
    unit_cell = Atoms(symbols=symbols, positions=positions, cell=cell, pbc=[True, True, False])
    # Create the supercell
    supercell = make_supercell(unit_cell, transformation_matrix)
    # Define the output filename
    filename = os.path.join(output_folder, f"supercell_{material['id']}.xyz")
    # Write the supercell to a file
    write(filename, supercell)
    print(f"Supercell expansion complete for {material['id']} and saved to '{filename}'")

print(f"Supercell expansion for all materials is complete and saved in the '{output_folder}' folder.")

In [ ]:
#Supercell generator: flatband with rotation

import os
import numpy as np
from ase.io import write
from ase.build import make_supercell
from ase import Atoms
import ast  # Use ast.literal_eval for safer evaluation
from scipy.spatial.transform import Rotation as R  # For rotation operations

SUPERCELL_SIZE = 3

transformation_matrix = np.array([[SUPERCELL_SIZE, 0, 0],
                                  [0, SUPERCELL_SIZE, 0],
                                  [0, 0, 1]])  # No replication in the c-direction

# Create a folder for saving the supercell files
output_folder = 'Supercells/supercells_flatband_rotated_3x3'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to parse the structures from the text file
def parse_structures(filename):
    import re
    materials = []
    with open(filename, 'r') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('2dm-'):
            material = {}
            material['id'] = line.strip()
            i += 1
            # Read formula and code
            line = lines[i].strip()
            formula_and_code = line.split()
            material['formula'] = formula_and_code[0]
            material['some_code'] = formula_and_code[1] if len(formula_and_code) > 1 else ''
            i += 1
            # Read lattice vectors
            if lines[i].strip() == 'lattice vector':
                i += 1
                material['lattice_vectors'] = ast.literal_eval(lines[i].strip())
                i += 1
            else:
                raise ValueError(f"Expected 'lattice vector' at line {i+1}")
            # Skip empty lines
            while i < len(lines) and not lines[i].strip():
                i += 1
            # Read headers
            if i < len(lines):
                headers = lines[i].strip().split()
                i += 1
            else:
                break
            # Read atom data
            atoms = []
            while i < len(lines):
                line = lines[i].strip()
                if not line or line.startswith('2dm-'):
                    break
                # Parse the atom line
                try:
                    atom_dict = parse_atom_line(line)
                    atoms.append(atom_dict)
                except ValueError as e:
                    print(f"Warning: {e}")
                i += 1
            material['atoms'] = atoms
            materials.append(material)
        else:
            i += 1
    return materials

def parse_atom_line(line):
    import re
    # Remove leading index if present
    line = line.strip()
    if line and line[0].isdigit():
        parts = line.split(None, 1)
        line = parts[1]
    # Now extract the label, which is the first word
    parts = line.split(None, 1)
    if len(parts) < 2:
        raise ValueError("Could not parse atom line: {}".format(line))
    label = parts[0]
    rest = parts[1]
    # Use regular expressions to find the content inside brackets or braces
    pattern = r'\[.*?\]|\{.*?\}|[^\s]+'
    matches = re.findall(pattern, rest)
    # The expected order: xyz, abc, species, properties
    if len(matches) >= 4:
        xyz_str = matches[0]
        abc_str = matches[1]
        species_str = matches[2]
        properties_str = matches[3]
        return {
            'label': label,
            'xyz': xyz_str,
            'abc': abc_str,
            'species': species_str,
            'properties': properties_str
        }
    else:
        raise ValueError("Could not parse atom line: {}".format(line))

# Parse the structures from the text file
materials = parse_structures('Materials/reduced_sublattice_structure.txt')

# Iterate over the parsed materials and generate supercells
for material in materials:
    # Extract lattice vectors and atom data
    cell = np.array(material['lattice_vectors'], dtype=float)  # Convert to numpy array
    atoms_data = material['atoms']
    symbols = []
    positions = []
    for atom in atoms_data:
        symbol = atom['label']
        position = np.array(ast.literal_eval(atom['xyz']), dtype=float)  # Safely convert string to array
        symbols.append(symbol)
        positions.append(position)
    positions = np.array(positions)  # Convert to numpy array

    # Get lattice vectors a and b
    a = cell[0]
    b = cell[1]

    # Check if the z-components of both a and b are zero (within a small tolerance)
    if np.isclose(a[2], 0.0, atol=1e-6) and np.isclose(b[2], 0.0, atol=1e-6):
        # No rotation needed
        rotation_matrix = np.eye(3)
    else:
        # Step 1: Compute the rotation matrix to align the lattice plane with the xy-plane
        # Compute the normal vector to the plane defined by a and b
        normal_vector = np.cross(a, b)
        normal_vector /= np.linalg.norm(normal_vector)  # Normalize

        # Compute the angle between the normal vector and the z-axis
        z_axis = np.array([0, 0, 1])
        dot_product = np.dot(normal_vector, z_axis)
        rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))

        # Compute rotation axis
        rotation_axis = np.cross(normal_vector, z_axis)
        if np.linalg.norm(rotation_axis) < 1e-6:
            rotation_axis = np.array([1.0, 0.0, 0.0])  # Arbitrary axis as float
        else:
            rotation_axis /= np.linalg.norm(rotation_axis)  # Normalize

        # Create the rotation matrix
        rotation = R.from_rotvec(rotation_angle * rotation_axis)
        rotation_matrix = rotation.as_matrix()

    # Step 2: Apply the rotation to cell and positions
    rotated_cell = (rotation_matrix @ cell.T).T  # Transpose back to (3, 3)
    rotated_positions = (rotation_matrix @ positions.T).T

    # Optional: Round small values to zero for better numerical stability
    rotated_cell[np.abs(rotated_cell) < 1e-8] = 0.0
    rotated_positions[np.abs(rotated_positions) < 1e-8] = 0.0

    # Step 3: Create the unit cell structure with rotated cell and positions
    unit_cell = Atoms(symbols=symbols, positions=rotated_positions, cell=rotated_cell, pbc=[True, True, False])

    # Create the supercell
    supercell = make_supercell(unit_cell, transformation_matrix)

    # Define the output filename
    filename = os.path.join(output_folder, f"rotated_supercell_{material['id']}.xyz")

    # Write the supercell to a file using 'extxyz' format
    write(filename, supercell, format='extxyz')
    print(f"Supercell expansion complete for {material['id']} and saved to '{filename}'")

print(f"Supercell expansion for all materials is complete and saved in the '{output_folder}' folder.")


In [ ]:
#Supercell generator: flatband with rotation and shifts with revised logic
import os
import numpy as np
from ase.io import write
from ase.build import make_supercell
from ase import Atoms
import ast
from scipy.spatial.transform import Rotation as R

SUPERCELL_SIZE = 3

transformation_matrix = np.array([[SUPERCELL_SIZE, 0, 0],
                                  [0, SUPERCELL_SIZE, 0],
                                  [0, 0, 1]])

# Create a folder for saving the supercell files
output_folder = 'Supercells/supercells_flatband_rotated_shifted_3x3'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to parse the structures from the text file
def parse_structures(filename):
    import re
    materials = []
    with open(filename, 'r') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('2dm-'):
            material = {}
            material['id'] = line.strip()
            i += 1
            # Read formula and code
            line = lines[i].strip()
            formula_and_code = line.split()
            material['formula'] = formula_and_code[0]
            material['some_code'] = formula_and_code[1] if len(formula_and_code) > 1 else ''
            i += 1
            # Read lattice vectors
            if lines[i].strip() == 'lattice vector':
                i += 1
                material['lattice_vectors'] = ast.literal_eval(lines[i].strip())
                i += 1
            else:
                raise ValueError(f"Expected 'lattice vector' at line {i+1}")
            # Skip empty lines
            while i < len(lines) and not lines[i].strip():
                i += 1
            # Read headers
            if i < len(lines):
                headers = lines[i].strip().split()
                i += 1
            else:
                break
            # Read atom data
            atoms = []
            while i < len(lines):
                line = lines[i].strip()
                if not line or line.startswith('2dm-'):
                    break
                # Parse the atom line
                try:
                    atom_dict = parse_atom_line(line)
                    atoms.append(atom_dict)
                except ValueError as e:
                    print(f"Warning: {e}")
                i += 1
            material['atoms'] = atoms
            materials.append(material)
        else:
            i += 1
    return materials

def parse_atom_line(line):
    import re
    # Remove leading index if present
    line = line.strip()
    if line and line[0].isdigit():
        parts = line.split(None, 1)
        line = parts[1]
    # Now extract the label, which is the first word
    parts = line.split(None, 1)
    if len(parts) < 2:
        raise ValueError("Could not parse atom line: {}".format(line))
    label = parts[0]
    rest = parts[1]
    # Use regular expressions to find the content inside brackets or braces
    pattern = r'\[.*?\]|\{.*?\}|[^\s]+'
    matches = re.findall(pattern, rest)
    # The expected order: xyz, abc, species, properties
    if len(matches) >= 4:
        xyz_str = matches[0]
        abc_str = matches[1]
        species_str = matches[2]
        properties_str = matches[3]
        return {
            'label': label,
            'xyz': xyz_str,
            'abc': abc_str,
            'species': species_str,
            'properties': properties_str
        }
    else:
        raise ValueError("Could not parse atom line: {}".format(line))


# Parse the structures from the text file
materials = parse_structures('Materials/reduced_sublattice_structure.txt')

# Iterate over the parsed materials and generate supercells
for material in materials:
    # Extract lattice vectors and atom data
    cell = np.array(material['lattice_vectors'], dtype=float)  # Convert to numpy array
    atoms_data = material['atoms']
    symbols = []
    positions = []
    for atom in atoms_data:
        symbol = atom['label']
        position = np.array(ast.literal_eval(atom['xyz']), dtype=float)  # Safely convert string to array
        symbols.append(symbol)
        positions.append(position)
    positions = np.array(positions)  # Convert to numpy array

    # Get lattice vectors a and b
    a = cell[0]
    b = cell[1]
    c_vector = cell[2]

    # Check if the z-components of both a and b are zero (within a small tolerance)
    if np.isclose(a[2], 0.0, atol=1e-6) and np.isclose(b[2], 0.0, atol=1e-6):
        # No rotation needed
        rotation_matrix = np.eye(3)
        print(f"No rotation applied for {material['id']} as z-components of a and b are zero.")
    else:
        # Step 1: Compute the rotation matrix to align the lattice plane with the xy-plane
        # Compute the normal vector to the plane defined by a and b
        normal_vector = np.cross(a, b)
        normal_vector /= np.linalg.norm(normal_vector)  # Normalize

        # Compute the angle between the normal vector and the z-axis
        z_axis = np.array([0, 0, 1])
        dot_product = np.dot(normal_vector, z_axis)
        rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))

        # Compute rotation axis
        rotation_axis = np.cross(normal_vector, z_axis)
        if np.linalg.norm(rotation_axis) < 1e-6:
            rotation_axis = np.array([1.0, 0.0, 0.0])  # Arbitrary axis as float
        else:
            rotation_axis /= np.linalg.norm(rotation_axis)  # Normalize

        # Create the rotation matrix
        rotation = R.from_rotvec(rotation_angle * rotation_axis)
        rotation_matrix = rotation.as_matrix()

        print(f"Rotation applied for {material['id']}.")

    # Step 2: Apply the rotation to cell and positions
    rotated_cell = (rotation_matrix @ cell.T).T  # Transpose back to (3, 3)
    rotated_positions = (rotation_matrix @ positions.T).T

    # Round small values to zero for better numerical stability
    rotated_cell[np.abs(rotated_cell) < 1e-8] = 0.0
    rotated_positions[np.abs(rotated_positions) < 1e-8] = 0.0

    # Step 3: Create the unit cell structure with rotated cell and positions
    unit_cell = Atoms(symbols=symbols, positions=rotated_positions, cell=rotated_cell, pbc=[True, True, False])

    # **New Step: Adjust Layers Along the c-axis**
    # Get scaled positions
    scaled_positions = unit_cell.get_scaled_positions()
    z_scaled = scaled_positions[:, 2]

    # Determine if layers are at the cell boundaries
    z_min = np.min(z_scaled)
    z_max = np.max(z_scaled)
    z_range = z_max - z_min

    # Define thresholds
    threshold_low = 0.25
    threshold_high = 0.75

    # Check if atoms are near z=0 or z=1
    adjust_needed = False
    if z_min < threshold_low and z_max > threshold_high:
        adjust_needed = True
        print(f"Adjustment needed for {material['id']} to center layers along the c-axis.")

        # Compute mean z position considering periodicity
        angles = 2 * np.pi * z_scaled
        complex_numbers = np.exp(1j * angles)
        mean_complex = np.mean(complex_numbers)
        mean_angle = np.angle(mean_complex)
        z_mean = mean_angle / (2 * np.pi)
        if z_mean < 0:
            z_mean += 1  # Ensure z_mean is within [0,1)

        # Compute shift to bring z_mean to 0.5
        shift = 0.5 - z_mean

        # Shift all atoms
        scaled_positions[:, 2] += shift

        # Ensure scaled positions are within [0,1)
        scaled_positions[:, 2] = np.mod(scaled_positions[:, 2], 1.0)

        # Update positions
        unit_cell.set_scaled_positions(scaled_positions)


    # Proceed to create the supercell
    supercell = make_supercell(unit_cell, transformation_matrix)

    # Define the output filename
    filename = os.path.join(output_folder, f"supercell_{material['id']}.xyz")

    # Write the supercell to a file using 'extxyz' format
    write(filename, supercell, format='extxyz')
    print(f"Supercell expansion complete for {material['id']} and saved to '{filename}'\n")

print(f"Supercell expansion for all materials is complete and saved in the '{output_folder}' folder.")

In [1]:
#Supercell generator: flatband with rotation and shifts with revised logic and rotated to align a and x
import os
import numpy as np
from ase.io import write
from ase.build import make_supercell
from ase import Atoms
import ast
from scipy.spatial.transform import Rotation as R

SUPERCELL_SIZE = 10

transformation_matrix = np.array([[SUPERCELL_SIZE, 0, 0],
                                  [0, SUPERCELL_SIZE, 0],
                                  [0, 0, 1]])

# Create a folder for saving the supercell files
output_folder = 'Supercells/supercells_flatband_rotated_shifted_aligned_10x10'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to parse the structures from the text file
def parse_structures(filename):
    import re
    materials = []
    with open(filename, 'r') as f:
        lines = f.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('2dm-'):
            material = {}
            material['id'] = line.strip()
            i += 1
            # Read formula and code
            line = lines[i].strip()
            formula_and_code = line.split()
            material['formula'] = formula_and_code[0]
            material['some_code'] = formula_and_code[1] if len(formula_and_code) > 1 else ''
            i += 1
            # Read lattice vectors
            if lines[i].strip() == 'lattice vector':
                i += 1
                material['lattice_vectors'] = ast.literal_eval(lines[i].strip())
                i += 1
            else:
                raise ValueError(f"Expected 'lattice vector' at line {i+1}")
            # Skip empty lines
            while i < len(lines) and not lines[i].strip():
                i += 1
            # Read headers
            if i < len(lines):
                headers = lines[i].strip().split()
                i += 1
            else:
                break
            # Read atom data
            atoms = []
            while i < len(lines):
                line = lines[i].strip()
                if not line or line.startswith('2dm-'):
                    break
                # Parse the atom line
                try:
                    atom_dict = parse_atom_line(line)
                    atoms.append(atom_dict)
                except ValueError as e:
                    print(f"Warning: {e}")
                i += 1
            material['atoms'] = atoms
            materials.append(material)
        else:
            i += 1
    return materials

def parse_atom_line(line):
    import re
    # Remove leading index if present
    line = line.strip()
    if line and line[0].isdigit():
        parts = line.split(None, 1)
        line = parts[1]
    # Now extract the label, which is the first word
    parts = line.split(None, 1)
    if len(parts) < 2:
        raise ValueError("Could not parse atom line: {}".format(line))
    label = parts[0]
    rest = parts[1]
    # Use regular expressions to find the content inside brackets or braces
    pattern = r'\[.*?\]|\{.*?\}|[^\s]+'
    matches = re.findall(pattern, rest)
    # The expected order: xyz, abc, species, properties
    if len(matches) >= 4:
        xyz_str = matches[0]
        abc_str = matches[1]
        species_str = matches[2]
        properties_str = matches[3]
        return {
            'label': label,
            'xyz': xyz_str,
            'abc': abc_str,
            'species': species_str,
            'properties': properties_str
        }
    else:
        raise ValueError("Could not parse atom line: {}".format(line))


# Parse the structures from the text file
materials = parse_structures('Materials/reduced_sublattice_structure.txt')

# Iterate over the parsed materials and generate supercells
for material in materials:
    # Extract lattice vectors and atom data
    cell = np.array(material['lattice_vectors'], dtype=float)  # Convert to numpy array
    atoms_data = material['atoms']
    symbols = []
    positions = []
    for atom in atoms_data:
        symbol = atom['label']
        position = np.array(ast.literal_eval(atom['xyz']), dtype=float)  # Safely convert string to array
        symbols.append(symbol)
        positions.append(position)
    positions = np.array(positions)  # Convert to numpy array

    # Get lattice vectors a and b
    a = cell[0]
    b = cell[1]
    c_vector = cell[2]

    # Check if the z-components of both a and b are zero (within a small tolerance)
    if np.isclose(a[2], 0.0, atol=1e-6) and np.isclose(b[2], 0.0, atol=1e-6):
        # No rotation needed
        rotation_matrix = np.eye(3)
        print(f"No rotation applied for {material['id']} as z-components of a and b are zero.")
        rotated_cell = cell.copy()
        rotated_positions = positions.copy()
    else:
        # Step 1: Compute the rotation matrix to align the lattice plane with the xy-plane
        # Compute the normal vector to the plane defined by a and b
        normal_vector = np.cross(a, b)
        normal_vector /= np.linalg.norm(normal_vector)  # Normalize

        # Compute the angle between the normal vector and the z-axis
        z_axis = np.array([0, 0, 1])
        dot_product = np.dot(normal_vector, z_axis)
        rotation_angle = np.arccos(np.clip(dot_product, -1.0, 1.0))

        # Compute rotation axis
        rotation_axis = np.cross(normal_vector, z_axis)
        if np.linalg.norm(rotation_axis) < 1e-6:
            rotation_axis = np.array([1.0, 0.0, 0.0])  # Arbitrary axis as float
        else:
            rotation_axis /= np.linalg.norm(rotation_axis)  # Normalize

        # Create the rotation matrix
        rotation = R.from_rotvec(rotation_angle * rotation_axis)
        rotation_matrix = rotation.as_matrix()

        print(f"Rotation applied for {material['id']} to align lattice plane with xy-plane.")

        # Apply the rotation to cell and positions
        rotated_cell = (rotation_matrix @ cell.T).T  # Transpose back to (3, 3)
        rotated_positions = (rotation_matrix @ positions.T).T

    # Step 2: Rotate around z-axis to align a along x-axis
    a_rotated = rotated_cell[0]
    a_xy = a_rotated[:2]  # components in x and y
    norm_a_xy = np.linalg.norm(a_xy)

    if norm_a_xy > 1e-6:
        theta = np.arctan2(a_xy[1], a_xy[0])  # angle between a_rotated and x-axis

        # Create rotation matrix for rotation about z-axis by angle -theta
        rotation_z = np.array([[np.cos(-theta), -np.sin(-theta), 0],
                               [np.sin(-theta),  np.cos(-theta), 0],
                               [            0,              0,    1]])

        # Apply this rotation to rotated_cell and rotated_positions
        rotated_cell = (rotation_z @ rotated_cell.T).T
        rotated_positions = (rotation_z @ rotated_positions.T).T

        print(f"Additional rotation applied around z-axis to align a along x-axis for {material['id']}.")
    else:
        print(f"Lattice vector a is already aligned with x-axis for {material['id']}.")

    # Round small values to zero for better numerical stability
    rotated_cell[np.abs(rotated_cell) < 1e-8] = 0.0
    rotated_positions[np.abs(rotated_positions) < 1e-8] = 0.0

    # Step 3: Create the unit cell structure with rotated cell and positions
    unit_cell = Atoms(symbols=symbols, positions=rotated_positions, cell=rotated_cell, pbc=[True, True, False])

    # **New Step: Adjust Layers Along the c-axis**
    # Get scaled positions
    scaled_positions = unit_cell.get_scaled_positions()
    z_scaled = scaled_positions[:, 2]

    # Determine if layers are at the cell boundaries
    z_min = np.min(z_scaled)
    z_max = np.max(z_scaled)
    z_range = z_max - z_min

    # Define thresholds
    threshold_low = 0.25
    threshold_high = 0.75

    # Check if atoms are near z=0 or z=1
    adjust_needed = False
    if z_min < threshold_low and z_max > threshold_high:
        adjust_needed = True
        print(f"Adjustment needed for {material['id']} to center layers along the c-axis.")

        # Compute mean z position considering periodicity
        angles = 2 * np.pi * z_scaled
        complex_numbers = np.exp(1j * angles)
        mean_complex = np.mean(complex_numbers)
        mean_angle = np.angle(mean_complex)
        z_mean = mean_angle / (2 * np.pi)
        if z_mean < 0:
            z_mean += 1  # Ensure z_mean is within [0,1)

        # Compute shift to bring z_mean to 0.5
        shift = 0.5 - z_mean

        # Shift all atoms
        scaled_positions[:, 2] += shift

        # Ensure scaled positions are within [0,1)
        scaled_positions[:, 2] = np.mod(scaled_positions[:, 2], 1.0)

        # Update positions
        unit_cell.set_scaled_positions(scaled_positions)


    # Proceed to create the supercell
    supercell = make_supercell(unit_cell, transformation_matrix)

    # Define the output filename
    filename = os.path.join(output_folder, f"supercell_{material['id']}.xyz")

    # Write the supercell to a file using 'extxyz' format
    write(filename, supercell, format='extxyz')
    print(f"Supercell expansion complete for {material['id']} and saved to '{filename}'\n")

print(f"Supercell expansion for all materials is complete and saved in the '{output_folder}' folder.")


No rotation applied for 2dm-21 as z-components of a and b are zero.
Additional rotation applied around z-axis to align a along x-axis for 2dm-21.
Supercell expansion complete for 2dm-21 and saved to 'Supercells/supercells_flatband_rotated_shifted_aligned_10x10\supercell_2dm-21.xyz'

Rotation applied for 2dm-22 to align lattice plane with xy-plane.
Additional rotation applied around z-axis to align a along x-axis for 2dm-22.
Supercell expansion complete for 2dm-22 and saved to 'Supercells/supercells_flatband_rotated_shifted_aligned_10x10\supercell_2dm-22.xyz'

No rotation applied for 2dm-25 as z-components of a and b are zero.
Additional rotation applied around z-axis to align a along x-axis for 2dm-25.
Supercell expansion complete for 2dm-25 and saved to 'Supercells/supercells_flatband_rotated_shifted_aligned_10x10\supercell_2dm-25.xyz'

Rotation applied for 2dm-29 to align lattice plane with xy-plane.
Additional rotation applied around z-axis to align a along x-axis for 2dm-29.
Adjust

In [ ]:
#Supercell viewer

from ase.io import read
from ase.visualize import view

# Specify the path to the supercell file you want to view
# Replace 'supercell_flatband/supercell_2dm-21.xyz' with the desired file
supercell_file = 'Supercells/supercells_flatband_1x1/supercell_2dm-22.xyz'

# Read the supercell structure
supercell = read(supercell_file)

# View the supercell structure
view(supercell)